In [1]:
import pandas as pd
import os
import random
import csv
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Take the large dataset and create a smaller subset of it.

input_file_path = '../data/flight-data.csv'
reduced_flight_data_path = '../data/reduced-flight-Data.csv'

if os.path.exists(reduced_flight_data_path):
    os.remove(reduced_flight_data_path)

# choose how many rows to copy
rows_to_copy = 6000

# Method 1: using csv reader and writer

with open(input_file_path, 'r') as input_file, open(reduced_flight_data_path, 'w', newline='') as output_file:
    csv_reader = csv.reader(input_file)
    csv_writer = csv.writer(output_file)

    header = next(csv_reader)
    csv_writer.writerow(header)

    all_rows = list(csv_reader)

    rows_to_copy = min(rows_to_copy, len(all_rows))

    random_rows = random.sample(all_rows, rows_to_copy)
    for row in random_rows:
        csv_writer.writerow(row)

print(f'Copied {rows_to_copy} random rows from {input_file_path} to {reduced_flight_data_path}')

Copied 6000 random rows from ../data/flight-data.csv to ../data/reduced-flight-Data.csv


In [3]:

original_data = pd.read_csv(input_file_path)
data = pd.read_csv(reduced_flight_data_path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   from_airport_code                6000 non-null   object 
 1   from_country                     6000 non-null   object 
 2   dest_airport_code                6000 non-null   object 
 3   dest_country                     6000 non-null   object 
 4   aircraft_type                    5923 non-null   object 
 5   airline_number                   6000 non-null   object 
 6   airline_name                     6000 non-null   object 
 7   flight_number                    6000 non-null   object 
 8   departure_time                   6000 non-null   object 
 9   arrival_time                     6000 non-null   object 
 10  duration                         6000 non-null   int64  
 11  stops                            6000 non-null   int64  
 12  price               

## Remove the irrelevant columns
We will try to redict the price based on the all the other columns.

We will remove the following columns:
- departure_time, arrival_time: The type is datetime, and we don't handle datetime columns
- currency: The currency doesn't have any effect on the other columns
- from_country, dest_country: The country doesn't have any effect on the other columns, and we can find the country from the airport code
- flight_number: The flight number doesn't have any effect on the price
- scan_date: The scan date doesn't have any effect on the price
- co2_emissions: at first sight, co2 emissions don't seem to have an effect on price
- co2_percentage: at first sight, co2 percentage don't seem to have an effect on price

We will keep the following columns:
- from_airport_code and dest_airport_code
- aircraft_type
- airline_number, airline_name
- duration
- stops
- price
- avg_co2_emission_for_this_route

In [4]:
data = data.drop(['departure_time', 'arrival_time', 'currency', 'from_country', 'dest_country', 'flight_number', 'scan_date', 'co2_emissions', 'co2_percentage','avg_co2_emission_for_this_route'], axis=1)

# 4. Fill missing values

In [5]:
pd.DataFrame({
    'Na amount': data.isna().sum(),
    'Na percentage': data.isna().sum()/data.shape[0]*100    
})

,Na amount,Na percentage
from_airport_code,0,0.000000
dest_airport_code,0,0.000000
aircraft_type,77,1.283333
airline_number,0,0.000000
airline_name,0,0.000000
duration,0,0.000000
stops,0,0.000000
price,13,0.216667


Here are all the columns with missing values and the way we will fill them:

| Column | Strategy | |
| :-- | :-: | :-- | 
| Aircraft Type | we can fill the missing values with the most common aircraft type for the given route, airline and number of stops, if the route is unique drop the row |
| Price | we drop these since they are needed for modeling  | |

In [6]:
for i, row in data[data['aircraft_type'].isnull()].iterrows():
    subset = data[(data['from_airport_code'] == row['from_airport_code']) & 
              (data['dest_airport_code'] == row['dest_airport_code']) & 
              (data['stops'] == row['stops']) &
              (data['airline_number'] == row['airline_number'])]

    if subset['aircraft_type'].isnull().sum() != subset.shape[0]:
        most_common_aircraft_type = subset['aircraft_type'].value_counts().idxmax()
        data.loc[i, 'aircraft_type'] = most_common_aircraft_type
        
        
data = data.dropna()

In [7]:
pd.DataFrame({
    'Na amount': data.isna().sum(),
    'Na percentage': data.isna().sum()/data.shape[0]*100    
})

,Na amount,Na percentage
from_airport_code,0,0.0
dest_airport_code,0,0.0
aircraft_type,0,0.0
airline_number,0,0.0
airline_name,0,0.0
duration,0,0.0
stops,0,0.0
price,0,0.0


In [8]:

deleted_rows = original_data.shape[0] - data.shape[0]
print(f'Deleted {deleted_rows} rows of the original data')

Deleted 992927 rows of the original data


## Fix the data types of the columns

In [9]:
data[['from_airport_code', 'dest_airport_code','aircraft_type','airline_number', 'airline_name']] = data[['from_airport_code', 'dest_airport_code','aircraft_type','airline_number', 'airline_name']].astype('category')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5939 entries, 0 to 5999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   from_airport_code  5939 non-null   category
 1   dest_airport_code  5939 non-null   category
 2   aircraft_type      5939 non-null   category
 3   airline_number     5939 non-null   category
 4   airline_name       5939 non-null   category
 5   duration           5939 non-null   int64   
 6   stops              5939 non-null   int64   
 7   price              5939 non-null   float64 
dtypes: category(5), float64(1), int64(2)
memory usage: 360.2 KB


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data['from_airport_code'] = label_encoder.fit_transform(data['from_airport_code'])
data['dest_airport_code'] = label_encoder.fit_transform(data['dest_airport_code'])
data['aircraft_type'] = label_encoder.fit_transform(data['aircraft_type'])
data['airline_number'] = label_encoder.fit_transform(data['airline_number'])
data['airline_name'] = label_encoder.fit_transform(data['airline_name'])

data['from_airport_code'] = label_encoder.fit_transform(data['from_airport_code'])
data['dest_airport_code'] = label_encoder.fit_transform(data['dest_airport_code'])
data['aircraft_type'] = label_encoder.fit_transform(data['aircraft_type'])
data['airline_number'] = label_encoder.fit_transform(data['airline_number'])
data['airline_name'] = label_encoder.fit_transform(data['airline_name'])

data.head()

,from_airport_code,dest_airport_code,aircraft_type,airline_number,airline_name,duration,stops,price
0,13,70,865,20,502,2145,2,1952.0
1,5,52,1879,43,813,730,1,825.0
2,12,30,707,70,1269,565,1,637.0
3,23,74,1066,83,784,1275,2,2747.0
4,4,70,1609,83,405,2036,3,1604.0


In [12]:
data.to_csv('../data/processed-flight-data.csv', index=False)